In [3]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("sagawa/ZINC-canonicalized")

train_df, valid_df = ds['train'].to_pandas(), ds['validation'].to_pandas()
full_df = pd.concat([train_df, valid_df], ignore_index=True).reset_index()
full_df['dataset'] = 'ZINC'
full_df = full_df[['smiles', 'dataset']]

# full_df.to_csv('./ZINC.csv', index=False)

full_df.to_parquet(
    './ZINC.parquet',
    engine='pyarrow',      # pyarrow 엔진 사용 (권장)
    compression='snappy',  # snappy 압축 사용 (속도와 압축률의 균형)
    index=False            # 인덱스 미포함
) # <-- 수정된 부분

dataset_infos.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

In [5]:
from rdkit import Chem
from tqdm import tqdm

def validate_smiles(smiles):
    """SMILES 문자열이 RDKit으로 파싱 가능한지 확인합니다."""
    if smiles is None or smiles == "":
        return False
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        # RDKit이 파싱 중 에러를 발생시키는 경우 (흔하지 않음)
        return False

def preprocess_and_filter_dataframe(df, smiles_column='smiles'):
    """
    데이터프레임에서 유효하지 않은 SMILES를 제거합니다.
    """
    print("Starting SMILES validation...")

    # tqdm을 pandas의 apply와 함께 사용하기 위해 설정
    tqdm.pandas(desc="Validating SMILES")

    # 각 SMILES에 대해 유효성 검사 함수 적용
    is_valid_mask = df[smiles_column].progress_apply(validate_smiles)

    # 유효한 데이터만 필터링
    filtered_df = df[is_valid_mask].copy()

    print(f"Original dataset size: {len(df)}")
    print(f"Filtered (Valid) dataset size: {len(filtered_df)}")
    print(f"Removed invalid SMILES: {len(df) - len(filtered_df)}")

    return filtered_df

cleaned_df = preprocess_and_filter_dataframe(full_df)
print(cleaned_df.shape)

Starting SMILES validation...


Validating SMILES: 100%|██████████| 22992522/22992522 [22:22<00:00, 17124.99it/s]


Original dataset size: 22992522
Filtered (Valid) dataset size: 22992521
Removed invalid SMILES: 1
(22992521, 2)


In [4]:
# cleaned_df.to_csv('./ZINC_cleaned.csv', index=False)

cleaned_df = pd.read_csv('./ZINC_cleaned.csv')

cleaned_df.to_parquet(
    './ZINC_cleaned.parquet',
    engine='pyarrow',
    compression='snappy',
    index=False
) # <-- 수정된 부분